# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [151]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [152]:
def answer_one():
    df = pd.read_csv('Employee_Movie_Choices.txt', sep='\t')
    G = nx.Graph()
    G.add_nodes_from(employees, bipartite=0)
    G.add_nodes_from(movies, bipartite=1)
    total_edges = []
    ind_edges = []
    for i in range(0, len(df)):
        ind_edges.append(df.iloc[i]['#Employee'])
        ind_edges.append(df.iloc[i]['Movie'])
        total_edges.append(tuple(ind_edges[:]))
        ind_edges.clear()
    G.add_edges_from(total_edges)
    
    
    return G

answer_one()

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [153]:
def answer_two():
    
    df = pd.read_csv('Employee_Movie_Choices.txt', sep='\t')
    G = nx.Graph()
    
    G.add_nodes_from(employees, bipartite=1)
    G.add_nodes_from(movies, bipartite=1)
    total_edges = []
    ind_edges = []
    for i in range(0, len(df)):
        ind_edges.append(df.iloc[i]['#Employee'])
        ind_edges.append(df.iloc[i]['Movie'])
        total_edges.append(tuple(ind_edges[:]))
        ind_edges.clear()
    G.add_edges_from(total_edges)
    for node in G.nodes():
        if node in employees:
            G.add_node(node, role='employee')
        if node in movies:
            G.add_node(node, role='movie')
            
    return G
    
answer_two()

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [154]:
def answer_three():
    
    return bipartite.weighted_projected_graph(answer_two(), employees)

answer_three()

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [155]:
def answer_four():
        
    G = answer_three() 
    weightdf = pd.DataFrame(G.edges(data=True), columns =['Name1', 'Name2', 'Weight']) 
    w2 = []
    for i in range (0, 11):
        w2.append(str(weightdf.iloc[i]['Weight'])[11:12])
    weightdf['Weight'] = w2
    weightdf_copy = weightdf.copy()
    weightdf_copy.columns = ['Name2', 'Name1', 'Weight']
    weightdf_final = pd.concat([weightdf, weightdf_copy]).reset_index(drop=True)
    
    relationdf = pd.read_csv('Employee_Relationships.txt', sep='\t', header=None)
    relationdf.columns = ['Name1', 'Name2', 'Relation']
    
    weight_relation_df = pd.merge(relationdf, weightdf_final, on = ['Name1', 'Name2'], how = 'left').fillna(0)
    weight_relation_df['Weight'] = pd.to_numeric(weight_relation_df['Weight'])
    
    return weight_relation_df['Weight'].corr(weight_relation_df['Relation'])

answer_four()

0.78839622217334737